In [ ]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
df = pd.read_csv("diabetes.csv")

In [ ]:
df.head()

## Exploratory  Data Analysis

In [ ]:
# for getting the shape (No of rows & columns)
df.shape

In [ ]:
# Column Details
df.columns

In [ ]:
# getting brief summary of data 
df.info()

In [ ]:
# Checking Null Values
df.isnull().sum()

In [ ]:
# Statistical summary of dataset
df.describe()

In [ ]:
df.corr()

In [ ]:
cor = df.corr()
sns.heatmap(cor, cmap="crest", annot=True)
plt.show()

In [ ]:
# Calculating the number of zero-values in in each column
print('rows having null Pregnancies value : {0}'.format(len(df.loc[df['Pregnancies'] == 0])))
print('rows having null Glucose value : {0}'.format(len(df.loc[df['Glucose'] == 0])))
print('rows having null BloodPressure value : {0}'.format(len(df.loc[df['BloodPressure'] == 0])))
print('rows having null SkinThickness value : {0}'.format(len(df.loc[df['SkinThickness'] == 0])))
print('rows having null Insulin value : {0}'.format(len(df.loc[df['Insulin'] == 0])))
print('rows having null BMI value : {0}'.format(len(df.loc[df['BMI'] == 0])))
print('rows having null DiabetesPedigreeFunction value : {0}'.format(len(df.loc[df['DiabetesPedigreeFunction'] == 0])))
print('rows having null Age value : {0}'.format(len(df.loc[df['Age'] == 0])))

In [ ]:
# Calculating the mean value of every column
means = df.iloc[:, 1:6].mean()
means

In [ ]:
nonzeros = list(df.columns[1:6])
for column in nonzeros:
    df[column] = df[column].replace(0, means[column])

In [ ]:
# Preview of Changed Dataset
df.head()

In [ ]:
outcome_counts = df['Outcome'].value_counts()
print(outcome_counts)

In [ ]:
df_improve =df[['Glucose','BMI','Age','Insulin','Outcome']]

In [ ]:
# improved dataset after datacleaning
df_improve.head()

In [ ]:
df_improve.corr()

## Data Visualisation


In [ ]:
# Heatmap
cor = df_improve.corr()
sns.heatmap(cor, cmap="crest", annot=True)
plt.show()

In [ ]:
# Pairplot
sns.pairplot(df_improve,hue='Outcome')

In [ ]:
# Histogram
features = df_improve.columns.tolist()
features.remove('Outcome')
# Create histograms for each feature with 'Outcome' as the hue
for feature in features:
    sns.histplot(data=df_improve, x=feature, hue='Outcome', kde=True)
    plt.title(f'Histogram of {feature}')
    plt.show()

In [ ]:
# Create a pie chart
plt.figure(figsize=(6, 6))
plt.pie(outcome_counts, labels=['Fit', 'Diabetic'], autopct='%1.1f%%', startangle=140)
plt.title('Outcome Count by Fit and Diabetic')
plt.show()

In [ ]:
# Plot the count of each class in the 'Outcome' column
sns.countplot(x=df_improve['Outcome'], data=df_improve)
plt.show()

## Feature Selection

In [ ]:
X = df_improve.loc[:, ['Glucose', 'Insulin','BMI','Age']].to_numpy()
y = df_improve.loc[:, 'Outcome'].to_numpy()

In [ ]:
print(X)

In [ ]:
print(y)

In [ ]:
#Standardising the Dataset
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

In [ ]:
scaler.fit(X)

In [ ]:
standardized_data = scaler.transform(X)

In [ ]:
print(standardized_data)

In [ ]:
X = standardized_data
target = df['Outcome']

In [ ]:
print(X)
print(y)

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.33, random_state = 42)

In [ ]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

## Decision Tree


In [ ]:
from collections import Counter

class Node:
    def __init__(self, feature=None, threshold=None, left=None, right=None,*,value=None):
        self.feature = feature
        self.threshold = threshold
        self.left = left
        self.right = right
        self.value = value
        
    def is_leaf_node(self):
        return self.value is not None

In [ ]:
class DecisionTree:
    def __init__(self, min_samples_split=2, max_depth=100, n_features=None):
        self.min_samples_split=min_samples_split
        self.max_depth=max_depth
        self.n_features=n_features
        self.root=None

    def fit(self, X, y):
        self.n_features = X.shape[1] if not self.n_features else min(X.shape[1],self.n_features)
        self.root = self._grow_tree(X, y)

    def _grow_tree(self, X, y, depth=0):
        n_samples, n_feats = X.shape
        n_labels = len(np.unique(y))

        # check the stopping criteria
        if (depth>=self.max_depth or n_labels==1 or n_samples<self.min_samples_split):
            leaf_value = self._most_common_label(y)
            return Node(value=leaf_value)

        feat_idxs = np.random.choice(n_feats, self.n_features, replace=False)

        # find the best split
        best_feature, best_thresh = self._best_split(X, y, feat_idxs)

        # create child nodes
        left_idxs, right_idxs = self._split(X[:, best_feature], best_thresh)
        left = self._grow_tree(X[left_idxs, :], y[left_idxs], depth+1)
        right = self._grow_tree(X[right_idxs, :], y[right_idxs], depth+1)
        return Node(best_feature, best_thresh, left, right)
    
    def _best_split(self, X, y, feat_idxs):
        best_gain = -1
        split_idx, split_threshold = None, None

        for feat_idx in feat_idxs:
            X_column = X[:, feat_idx]
            thresholds = np.unique(X_column)

            for thr in thresholds:
                # calculate the information gain
                gain = self._information_gain(y, X_column, thr)

                if gain > best_gain:
                    best_gain = gain
                    split_idx = feat_idx
                    split_threshold = thr

        return split_idx, split_threshold
    
    def _information_gain(self, y, X_column, threshold):
        # parent entropy
        parent_entropy = self._entropy(y)

        # create children
        left_idxs, right_idxs = self._split(X_column, threshold)

        if len(left_idxs) == 0 or len(right_idxs) == 0:
            return 0
        
        # calculate the weighted avg. entropy of children
        n = len(y)
        n_l, n_r = len(left_idxs), len(right_idxs)
        e_l, e_r = self._entropy(y[left_idxs]), self._entropy(y[right_idxs])
        child_entropy = (n_l/n) * e_l + (n_r/n) * e_r

        # calculate the IG
        information_gain = parent_entropy - child_entropy
        return information_gain

    def _split(self, X_column, split_thresh):
        left_idxs = np.argwhere(X_column <= split_thresh).flatten()
        right_idxs = np.argwhere(X_column > split_thresh).flatten()
        return left_idxs, right_idxs

    def _entropy(self, y):
        hist = np.bincount(y)
        ps = hist / len(y)
        return -np.sum([p * np.log(p) for p in ps if p>0])
    
    def _most_common_label(self, y):
        counter = Counter(y)
        value = counter.most_common(1)[0][0]
        return value

    def predict(self, X):
        return np.array([self._traverse_tree(x, self.root) for x in X])

    def _traverse_tree(self, x, node):
        if node.is_leaf_node():
            return node.value

        if x[node.feature] <= node.threshold:
            return self._traverse_tree(x, node.left)
        return self._traverse_tree(x, node.right)
        

In [ ]:
clf_DT = DecisionTree(max_depth=10)


In [ ]:
clf_DT.fit(X_train, y_train)


In [ ]:
y_pred_DT = clf_DT.predict(X_test)


In [ ]:
def accuracy(y_test, y_pred_DT):
    return np.sum(y_test == y_pred_DT) / len(y_test)

acc = accuracy(y_test, y_pred_DT)
print(acc)

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score


In [ ]:
def evaluate_model_performance(y_test, y_pred_DT):
    test_accuracy = accuracy_score(y_test, y_pred_DT)
    test_precision = precision_score(y_test, y_pred_DT)
    test_recall = recall_score(y_test, y_pred_DT)
    test_f1 = f1_score(y_test, y_pred_DT)

    print("Test Accuracy:", test_accuracy)
    print("Test Precision:", test_precision)
    print("Test Recall:", test_recall)
    print("Test F1 Score:", test_f1)

In [ ]:
evaluate_model_performance(y_test, y_pred_DT)

In [ ]:
from sklearn import metrics

In [ ]:
from sklearn.metrics import confusion_matrix

cm = confusion_matrix(y_test, y_pred_DT)

In [ ]:
print(cm)

In [ ]:
cm = metrics.confusion_matrix(y_test, y_pred_DT)

# You have a typo in the following line. Instead of "pd.df(cfn_matrix)", you should use "pd.DataFrame(cfn_matrix)".
sns.heatmap(pd.DataFrame(cm), annot=True, cmap="YlGnBu", fmt='g')
plt.title('Confusion Matrix', y=1.1)
plt.ylabel('Actual Label')
plt.xlabel('Predicted Label')
plt.show()

In [ ]:
# Classification Report
from sklearn.metrics import classification_report

print(classification_report(y_test,y_pred_DT))